In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
import gc
import os
import warnings
import absl.logging

# Suppress TensorFlow and other unnecessary logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Filter out INFO and WARNING logs
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["JAX_LOG_LEVEL"] = "ERROR"
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings("ignore")
absl.logging.set_verbosity(absl.logging.ERROR)

# TPU setup
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
except ValueError:
    strategy = tf.distribute.get_strategy()  # Default strategy for CPU and single GPU
    print("Running on CPU or GPU")

# Load data
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
sample_submission = pd.read_csv('/kaggle/input/ieee-fraud-detection/sample_submission.csv')

# Merge transaction and identity datasets
train = train_transaction.merge(train_identity, on='TransactionID', how='left')
test = test_transaction.merge(test_identity, on='TransactionID', how='left')

# Clean up memory
del train_transaction, train_identity, test_transaction, test_identity
gc.collect()

# Preprocessing function
def preprocess_data(train, test):
    # Label encode categorical features
    label_encoders = {}
    for col in train.select_dtypes(include=['object']).columns:
        if col in test.columns:
            le = LabelEncoder()
            combined_data = pd.concat([train[col], test[col]], axis=0).astype(str)
            le.fit(combined_data)
            train[col] = le.transform(train[col].astype(str))
            test[col] = le.transform(test[col].astype(str))
            label_encoders[col] = le

    # One-hot encode features with many categories
    train = pd.get_dummies(train, drop_first=True)
    test = pd.get_dummies(test, drop_first=True)
    
    # Align train and test data
    train, test = train.align(test, join='left', axis=1, fill_value=0)
    
    # Fill missing values
    train.fillna(-1, inplace=True)
    test.fillna(-1, inplace=True)
    
    # Remove special characters from feature names to avoid errors
    train.columns = train.columns.str.replace(r'[^A-Za-z0-9_]', '_', regex=True)
    test.columns = test.columns.str.replace(r'[^A-Za-z0-9_]', '_', regex=True)
    
    # Scale numeric features
    scaler = StandardScaler()
    numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
    train[numeric_cols] = scaler.fit_transform(train[numeric_cols])
    test[numeric_cols] = scaler.transform(test[numeric_cols])

    return train, test, label_encoders, scaler

# Preprocess data
train, test, label_encoders, scaler = preprocess_data(train, test)

# Check and clean target variable (isFraud) for unexpected values
print("Unique values in 'isFraud' before cleaning:", train['isFraud'].unique())
train['isFraud'] = train['isFraud'].replace({5: 1})  # Replace 5 with 1 if 5 represents fraud
print("Unique values in 'isFraud' after cleaning:", train['isFraud'].unique())

# Convert target variable to integer if not already in integer format
train['isFraud'] = train['isFraud'].astype(int)

# Separate target and features
X = train.drop(['isFraud', 'TransactionID'], axis=1)
y = train['isFraud']
X_test = test.drop(['TransactionID'], axis=1)

# Convert to NumPy arrays and ensure compatibility with TPU operations
X = np.array(X).astype('float32')
X_test = np.array(X_test).astype('float32')
y = np.array(y).astype('int32')

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network model inside TPU strategy scope
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])

# Calculate batch size based on strategy
batch_size = 1024
if hasattr(strategy, "num_replicas_in_sync") and strategy.num_replicas_in_sync:
    batch_size *= strategy.num_replicas_in_sync

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=batch_size,  # Adjust batch size for TPU if available
    verbose=1
)

# Predict on validation set and evaluate
val_preds = model.predict(tf.convert_to_tensor(X_val)).numpy().ravel()
roc_score = roc_auc_score(y_val, val_preds)
print(f'Validation ROC-AUC score: {roc_score}')

# Predict on test set for submission
test_preds = model.predict(tf.convert_to_tensor(X_test)).numpy().ravel()

# Create submission file
sample_submission['isFraud'] = test_preds
sample_submission.to_csv('submission.csv', index=False)
